# Running TCAV

This notebook walks you through an example application of the TCAV library step-by-step to understand which human interpretable concepts (e.g. stripes, dots, zigzags) are important to the image classifier GoogleNet's (a.k.a. Inception v1) prediction of Zebras.

## Install required packages

To run through this notebook in the interim, you are encouraged to utilize a `virtualenv` or `conda` environment for installing and working with the required packages to avoid any dependency and compatability issues with different versions of packages.

In [ ]:
import matplotlib.pyplot as plt
%pip install tcav

## Download example models and images

Open a terminal and run the following commands:

```
cd tcav/tcav_examples/image_models/imagenet

python download_and_make_datasets.py --source_dir=YOUR_PATH --number_of_images_per_folder=50 --number_of_random_folders=3
```

This script will download the following content into separate folders into a directory you specify with the `--source_dir=` argument:

**Images**
*  ImageNet images for the target Zebra class
*  [Broden dataset](http://netdissect.csail.mit.edu/) images for three concepts (e.g. striped, dotted, zigzagged)
*  Random ImageNet class images used by TCAV for hypothesis testing of important concepts

**Models**
*  [Inception 5h model](https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/inception5h.py)
*  [Mobilenet V2 model](https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet_v1.md)

## Import extensions and libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import json

from torchvision import models

from tcav.model import ModelWrapper
from tcav.dataset import JsonDataset
import tcav.activation_generator as act_gen
import tcav.cav as cav
import tcav.tcav as tcav
import tcav.utils as utils
import tcav.utils_plot as utils_plot # utils_plot requires matplotlib

## TCAV step-by-step

You will walk through the following steps below:

1. **Store example images in each folder** (you have this if you ran the above)
 * images for each concept
 * images for the class/labels of interest
 * random images that will be negative examples when learning CAVs (images that probably don't belong to any concepts)
2. **Write a model wrapper** (below uses example from tcav/model.py)
 * an instance of  ModelWrapper abstract class (in model.py). This tells TCAV class (tcav.py) how to communicate with your model (e.g., getting internal tensors)
3. **Retrieve model activations** (below uses example from tcav/activation_generator.py)
 * an instance of ActivationGeneratorInterface that tells TCAV class how to load example data and how to get activations from the model
4. Run TCAV and visualize scores for important concepts.

## Step 1: Store concept and target class images to local folders

... and tell TCAV where they are.

**source_dir**: where images of concepts, target class and random images (negative samples when learning CAVs) live. Each should be a sub-folder within this directory.

Note that random image directories can be in any name. In this example, we are using `random500_0`, `random500_1`,.. for an arbitrary reason. 

You need roughly 50-200 images per concept and target class (10-20 pictures also tend to work, but 200 is pretty safe).


**cav_dir**: directory to store CAVs (`None` if you don't want to store)

**target, concept**: names of the target class (that you want to investigate) and concepts (strings) - these are folder names in source_dir

**bottlenecks**: list of bottleneck names (intermediate layers in your model) that you want to use for TCAV. These names are defined in the model wrapper below.



In [ ]:
print ('REMEMBER TO UPDATE YOUR_PATH (where images, models are)!')

# This is the name of your model wrapper (InceptionV3 and GoogleNet are provided in model.py)
model_to_run = 'ResNet50'
user = 'angus'
# the name of the parent directory that results are stored (only if you want to cache)
project_name = 'tcav_zebra_test'
#working_dir = Path("/tmp/" + user + '/' + project_name)
working_dir = Path("/home/lina3782/labs/explain/imagenet/output")
# where activations are stored (only if your act_gen_wrapper does so)
activation_dir =  working_dir / 'activations'
# where CAVs are stored. 
# You can say None if you don't wish to store any.
cav_dir = working_dir / 'cavs'
grad_dir = working_dir / "grads"
# where the images live.

# TODO: replace 'YOUR_PATH' with path to downloaded models and images.
source_dir = Path('/home/lina3782/labs/explain/imagenet')
bottlenecks = ["layer2.1", "layer2.2", "layer2.3", "layer3.0", "layer3.1", "layer3.2", "layer3.3", "layer3.4", "layer3.5", "layer4.0", 'layer4.1', "layer4.2"]  # @param
bottlenecks = {bn:bn for bn in bottlenecks}

for d in [activation_dir, working_dir, cav_dir, grad_dir]:
    d.mkdir(exist_ok=True, parents=True)

# this is a regularizer penalty parameter for linear classifier to get CAVs. 
alphas = [0.1]   

num_random_exp = 30
target = 'zebra'
# concepts = ["polka", "flecked", "dotted", "striped", "lined", "banded", "zigzagged"]
concepts = ["striped"]

## Step 2: Write your model wrapper

The next step is to tell TCAV how to communicate with your model. See `model.GoogleNetWrapper_public ` for details.

You can define a subclass of ModelWrapper abstract class to do this. Let me walk you thru what each function does (tho they are pretty self-explanatory).  This wrapper includes a lot of the functions that you already have, for example, `get_prediction`.

### 2.1: Tensors from the graph: bottleneck tensors and ends
First, store your bottleneck tensors in `self.bottlenecks_tensors` as a dictionary. You only need bottlenecks that you are interested in running TCAV with. Similarly, fill in `self.ends` dictionary with `input`, `logit` and `prediction` tensors.

### 2.2: Define loss
Get your loss tensor, and assigned it to `self.loss`. This is what TCAV uses to take directional derivatives. 

While doing so, you would also want to set 
```python
self.y_input 
```
this simply is a tensorflow place holder for the target index in the logit layer (e.g., 0 index for a dog, 1 for a cat).
For multi-class classification, typically something like this works:

```python
self.y_input = tf.placeholder(tf.int64, shape=[None])
```

For example, for a multiclass classifier, something like below would work. 

```python
    # Construct gradient ops.
    with g.as_default():
      self.y_input = tf.placeholder(tf.int64, shape=[None])

      self.pred = tf.expand_dims(self.ends['prediction'][0], 0)

      self.loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
              labels=tf.one_hot(self.y_input, len(self.labels)),
              logits=self.pred))
    self._make_gradient_tensors()
```

### 2.3: Call _make_gradient_tensors in __init__() of your wrapper
```python
_make_gradient_tensors()  
```
does what you expect - given the loss and bottleneck tensors defined above, it adds gradient tensors.

### 2.4: Fill in labels, image shapes and a model name.
Get the mapping from labels (strings) to indice in the logit layer (int) in a dictionary format.

```python
def id_to_label(self, idx)
def label_to_id(self, label)
```

Set your input image shape at  `self.image_shape`


Set your model name to `self.model_name`

You are done with writing the model wrapper! See the two example model wrapers, InceptionV3 and Googlenet in `tcav/model.py`.

In [ ]:
def create_model():
    return models.resnet50(pretrained=True)

In [ ]:

# model_path is where the trained model is stored.
#model_path = source_dir / "/inception5h/example_model.pt"
model = create_model()
#model = model.load_state(model_path)

# LABEL_PATH is where the labels are stored. Each line contains one class, and they are ordered with respect to their index in 
# the logit layer. (yes, id_to_label function in the model wrapper reads from this file.)
# For example, imagenet_comp_graph_label_strings.txt looks like:
# dummy                                                                                      
# kit fox
# English setter
# Siberian husky ...
label_path = source_dir / "class_names.txt"
with open(label_path, "r") as fp:
    class_names = fp.read()
class_names = class_names.split("\n")
class_names_short = [v.split(",")[0] for v in class_names]
mymodel = ModelWrapper(model, bottlenecks, class_names_short)

In [ ]:
model

In [ ]:
class_names

In [ ]:
class_names_short

In [ ]:
mymodel.label_to_id(target)

## Step 3: Implement a class that returns activations (maybe with caching!)

Lastly, you will implement a class of the ActivationGenerationInterface which TCAV uses to load example data for a given concept or target, call into your model wrapper and return activations. I pulled out this logic outside of mymodel because this step often takes the longest. By making it modular, you can cache your activations and/or parallelize your computations, as I have done in `ActivationGeneratorBase.process_and_load_activations` in `activation_generator.py`.


The `process_and_load_activations` method of the activation generator must return a dictionary of activations that has concept or target name as  a first key, and the bottleneck name as a second key. So something like:

```python
{concept1: {bottleneck1: [[0.2, 0.1, ....]]},
concept2: {bottleneck1: [[0.1, 0.02, ....]]},
target1: {bottleneck1: [[0.02, 0.99, ....]]}
```


In [ ]:
data_path = source_dir / "data"

source_json = {}
for concept in [target] + concepts + [f"random500_{i}" for i in range(num_random_exp)]:
    paths = (data_path / concept).glob("*.jpg")
    source_json[concept] = {i: {"path": f"{concept}/{path.name}", "label": 0} for i, path in enumerate(paths)}

source_json_path = source_dir / "example_source_json.json"
with open(source_json_path, "w") as fp:
    json.dump(source_json, fp, indent=2)

source_json

concept_dict = {
    "concept1": {"0": {"path": "dasd", "label": "asd"}, "1": {}},
    "concept2": {},
    "random500_0": {},
    "random500_1": {}
}

In [ ]:
max_examples = 100
prefix = str(data_path) + "/"
num_workers = 4

act_generator = act_gen.ActivationGenerator(mymodel, source_json_path, activation_dir, JsonDataset, max_examples=max_examples, prefix=prefix, num_workers=4)

In [ ]:
example_dataset = JsonDataset(source_json, target, prefix)
example_dataset[0]

## Step 4: Run TCAV and visualize concept importance

You are now ready to run TCAV! Let's do it.

**num_random_exp**: number of experiments to confirm meaningful concept direction. TCAV will search for this many folders named `random500_0`, `random500_1`, etc. You can alternatively set the `random_concepts` keyword to be a list of folders of random concepts. Run at least 10-20 for meaningful tests. 

**random_counterpart**: as well as the above, you can optionally supply a single folder with random images as the "positive set" for statistical testing. Reduces computation time at the cost of less reliable random TCAV scores.


In [ ]:
## only running num_random_exp = 10 to save some time. The paper number are reported for 500 random runs.

mytcav = tcav.TCAV(
    target,
    concepts,
    bottlenecks,
    act_generator,
    alphas,
    cav_dir=cav_dir,
    num_random_exp=num_random_exp,
    do_random_pairs=True,
    grad_dir=grad_dir
)

In [ ]:
print("Training CAVs...")
mytcav.train_cavs(overwrite=False)


In [ ]:
print ('This may take a while... Go get coffee!')
results = mytcav.run(overwrite=False)
print ('done!')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = utils_plot.plot_results(results, num_random_exp=num_random_exp, figsize=(10, 5))
fig.axes[0].axhline(0.5, color="gray", alpha=0.8, linestyle="--")
fig

In [ ]:
results_dir = source_dir / f"results/{target}"
results_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
exp_name = "21-22-33-34-35-40-41-42_100.json"
with open(results_dir / f"{exp_name}.json", "w") as fp:
    json.dump(results, fp, indent=2)

In [ ]:
exp_name = "21-22-33-34-35-40-41-42_100.json"
with open(results_dir / f"{exp_name}.json", "r") as fp:
    results = json.load(fp)

In [ ]:
target = "zebra"
source_json = {}
for concept in [target] + concepts + [f"random500_{i}" for i in range(num_random_exp)]:
    paths = (data_path / concept).glob("*.jpg")
    source_json[concept] = {i: {"path": f"{concept}/{path.name}", "label": 0} for i, path in enumerate(paths)}

source_json_path = source_dir / "example_source_json.json"
with open(source_json_path, "w") as fp:
    json.dump(source_json, fp, indent=2)

act_generator = act_gen.ActivationGenerator(mymodel, source_json_path, activation_dir, JsonDataset, max_examples=max_examples, prefix=prefix, num_workers=4)

mytcav = tcav.TCAV(
    target,
    concepts,
    bottlenecks,
    act_generator,
    alphas,
    cav_dir=cav_dir,
    num_random_exp=num_random_exp,
    do_random_pairs=True,
    grad_dir=grad_dir
)
results = mytcav.run(overwrite=False)
fig = utils_plot.plot_results(results, num_random_exp=num_random_exp, figsize=(10, 5))
fig.axes[0].axhline(0.5, color="gray", alpha=0.8, linestyle="--")
fig

In [ ]:
import numpy as np
from PIL import Image

In [ ]:
num_cols = 7
num_rows = 2

num_img = num_rows * num_cols


for concept in concepts:
    print(concept)
    paths = [v["path"] for v in source_json[concept].values()]
    paths.sort()
    idx = np.random.choice(range(len(paths)), num_img, replace=False)
    paths = [data_path / paths[i] for i in idx]

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(2*num_cols, 2*num_rows))
    for i, ax in enumerate(axes.flatten()):
        img = Image.open(paths[i])
        img = np.array(img)
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()


In [ ]:
results[0].keys()

In [ ]:
import numpy as np

In [ ]:
all_acc_means = {}
all_acc_stds = {}
for concept in concepts:
    accs = {}
    for bn in bottlenecks.keys():
        accs[bn] = [v["cav_accuracies"]["overall"] for v in results if
                    (v["bottleneck"] == bn) and (v["cav_concept"] == concept)]
    accs = np.array(list(accs.values()))
    accs_mean = accs.mean(axis=1)
    accs_std = accs.std(axis=1)

    all_acc_means[concept] = accs_mean
    all_acc_stds[concept] = accs_std

fig, ax = plt.subplots()
for concept in concepts:
    ax.plot(list(bottlenecks.keys()), all_acc_means[concept], label=concept)
ax.set_xlabel("Layer")
ax.set_ylabel("Accuracy")
ax.tick_params(axis='x', rotation=45)
plt.legend(frameon=False)
plt.show()